In [98]:
# Importing necessary libraries
from transformers import LayoutLMForTokenClassification, LayoutLMTokenizerFast
from transformers import TrainingArguments, Trainer
from loader import *
import json
import torch

ModuleNotFoundError: No module named 'paddleocr'

In [69]:
# Loading the pre-trained LayoutLM model and tokenizer
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased")
tokenizer = LayoutLMTokenizerFast.from_pretrained("microsoft/layoutlm-base-uncased")


Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
# 读取JSON文件
data = dataSet('data/01.json',processor)
dataload = torch.utils.data.DataLoader(data,batch_size=2)

NameError: name 'dataSet' is not defined

In [94]:
data

[{'ocr': '/data/upload/6/e1b02bcd-01.png',
  'id': 1,
  'bbox': [{'x': 4.786641929499072,
    'y': 2.5174825174825175,
    'width': 53.02126444983588,
    'height': 6.8531468531468525,
    'rotation': 0,
    'original_width': 882,
    'original_height': 774},
   {'x': 29.210789210789205,
    'y': 23.916083916083917,
    'width': 45.779934351362925,
    'height': 5.734265734265733,
    'rotation': 0,
    'original_width': 882,
    'original_height': 774}],
  'label': [{'x': 4.786641929499072,
    'y': 2.5174825174825175,
    'width': 53.02126444983588,
    'height': 6.8531468531468525,
    'rotation': 0,
    'labels': ['Label 2'],
    'original_width': 882,
    'original_height': 774},
   {'x': 29.210789210789205,
    'y': 23.916083916083917,
    'width': 45.779934351362925,
    'height': 5.734265734265733,
    'rotation': 0,
    'labels': ['Label 1'],
    'original_width': 882,
    'original_height': 774}],
  'transcription': ['Full name: PETER ALLAN NOAD',
   'Email: noadies@xtra.co.n

In [76]:
text_inputs = []
for item in data:
    for annotation in item.get('annotations', []):
        for result in annotation.get('result', []):
            if 'labels' in result.get('value', {}):
                labels_text = ' '.join(result['value']['labels'])  # 将标签信息连接成文本
                text_inputs.append(labels_text)

# 将文本输入编码成模型可接受的格式
inputs = tokenizer(text_inputs, return_tensors="pt", padding=True, truncation=True)


In [80]:
# 创建训练数据集和评估数据集
# 这里可能需要根据数据量和训练需求进行拆分
train_dataset = inputs[:2]
eval_dataset = inputs[2:]

In [84]:
train_dataset

[Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [85]:
eval_dataset

[Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [81]:
# Setting up the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the training results
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps
    weight_decay=0.01,  # Weight decay coefficient
    logging_dir="./logs",  # Directory to save logs
)

# 使用trainer训练模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)


In [82]:
# 开始训练
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: 'tokenizers.Encoding' object has no attribute 'keys'